In [1]:
# %%capture
# Install the required libraries
%pip install --upgrade pip
%pip install uv
!uv pip install --system --no-progress --link-mode=symlink --index=https://download.pytorch.org/whl/cu124 accelerate unsloth vllm sentencepiece protobuf datasets wandb huggingface_hub kagglehub[hf-datasets] hf_transfer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Using Python 3.11.11 environment at: /usr
Resolved 187 packages in 1.97s
 Downloaded tokenizers
 Downloaded torchaudio
 Downloaded xgrammar
 Downloaded uvloop
 Downloaded torchvision
 Downloaded mistral-common
 Downloaded nvidia-nvjitlink-cu12
 Downloaded llguidance
 Downloaded nvidia-curand-cu12
 Downloaded pycountry
 Downloaded llvmlite
 Downloaded hf-xet
 Downloaded numba
 Downloaded xformers
 Downloaded bitsandbytes
 Downloaded nvidia-cusolver-cu12
 Downloaded vllm
 Downloaded nvidia-cusparselt-cu12
 Downloaded triton
 Downloaded nvidia-cusparse-cu12
 Downloa

In [2]:
!git clone https://github.com/shuhanmirza/Bengali-Poem-Dataset.git

Cloning into 'Bengali-Poem-Dataset'...
remote: Enumerating objects: 18637, done.
remote: Counting objects: 100% (18637/18637), done.
remote: Compressing objects: 100% (18269/18269), done.
remote: Total 18637 (delta 70), reused 18632 (delta 69), pack-reused 0 (from 0)
Receiving objects: 100% (18637/18637), 6.11 MiB | 16.29 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [3]:
import os
import random
import re
import json


def normalize_text(text):
    text = re.sub(r"\([\s\u09E6-\u09EF]+\)", " ", text)
    text = re.sub(r"[\u09E6-\u09EF]+", " ", text)
    text = re.sub(r"[\u00A0\u200B\u200C\u200D\u2060\u3000]+", " ", text)
    text = re.sub(r"[ \t\r\f\v]+", " ", text)
    return text.strip()


def make_datasets():
    # parent_dir is the root dataset directory
    # Assuming you ran (from project root)
    # cd ..
    # git clone https://github.com/shuhanmirza/Bengali-Poem-Dataset
    # It would be
    parent_dir = "/kaggle/working/Bengali-Poem-Dataset/dataset"

    poems = []
    classes = []
    missing_class_count = 0

    for poet in os.listdir(parent_dir):
        poet_dir = os.path.join(parent_dir, poet)
        if not os.path.isdir(poet_dir):
            continue

        for poem in os.listdir(poet_dir):
            poem_dir = os.path.join(poet_dir, poem)
            if not os.path.isdir(poem_dir):
                continue

            class_text = ""
            class_path = os.path.join(poem_dir, "CLASS.txt")
            if os.path.exists(class_path):
                try:
                    with open(class_path, "r", encoding="utf-8") as f:
                        class_text = normalize_text(f.read())
                except Exception as e:
                    print(f"Error reading CLASS.txt in {poem_dir}: {e}")
                    continue

            for file in os.listdir(poem_dir):
                file_path = os.path.join(poem_dir, file)
                if (
                    file.endswith(".txt")
                    and file not in ["CLASS.txt", "SOURCE.txt"]
                    and os.path.isfile(file_path)
                ):
                    try:
                        with open(file_path, "r", encoding="utf-8") as f:
                            poem_text = f.read()

                        poem_lines = []
                        for line in poem_text.split("\n"):
                            line = normalize_text(line)
                            if line:
                                poem_lines.append(line)

                        poem_text = "\n".join(poem_lines)

                        if class_text:
                            classes.append(
                                {
                                    "Instructions": f"একটি বাংলা কবিতা লেখো যার বিষয় হলো {class_text}। কবিতায় নতুন লাইনের জন্য '\n' এবং নতুন স্তবকের জন্য '\n\n\n' ব্যবহার করো।",
                                    "Input": class_text,
                                    "Output": poem_text,
                                }
                            )
                        else:
                            missing_class_count += 1

                        if len(poem_lines) >= 2:
                            random_line_start = random.randint(0, len(poem_lines) - 2)
                            line1 = poem_lines[random_line_start]
                            line2 = poem_lines[random_line_start + 1]

                            poems.append(
                                {
                                    "Instructions": f"নিচের দুটি লাইন ব্যবহার করে একটি সম্পূর্ণ বাংলা কবিতা লেখো। কবিতায় নতুন লাইনের জন্য '\n' এবং নতুন স্তবকের জন্য '\n\n\n' ব্যবহার করো।\n{poem_text}",
                                    "Input": f"{line1}\n{line2}",
                                    "Output": poem_text,
                                }
                            )

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
                        continue

    with open("/content/poems.json", "w", encoding="utf-8") as f:
        json.dump(poems, f, ensure_ascii=False, indent=4)
    with open("/content/classes.json", "w", encoding="utf-8") as f:
        json.dump(classes, f, ensure_ascii=False, indent=4)


if __name__ == "__main__":
    make_datasets()

In [4]:
import json
import os
import torch
import random


def split_dataset(
    input_file, train_output_file, test_output_file, test_size=0.2, random_state=42
):
    """
    Split a dataset into training and testing sets using PyTorch.

    Args:
        input_file (str): Path to the input JSON file
        train_output_file (str): Path to save the training data
        test_output_file (str): Path to save the testing data
        test_size (float): Proportion of the dataset to include in the test split
        random_state (int): Random seed for reproducibility
    """
    # Set seed for reproducibility
    torch.manual_seed(random_state)
    random.seed(random_state)

    # Load the data
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Calculate split sizes
    dataset_size = len(data)
    test_count = int(dataset_size * test_size)
    train_count = dataset_size - test_count

    # Create a random split with PyTorch
    indices = torch.randperm(dataset_size).tolist()
    train_indices = indices[:train_count]
    test_indices = indices[train_count:]

    # Create train and test datasets
    train_data = [data[i] for i in train_indices]
    test_data = [data[i] for i in test_indices]

    # Save the training data
    with open(train_output_file, "w", encoding="utf-8") as f:
        json.dump(train_data, f, ensure_ascii=False, indent=4)

    # Save the testing data
    with open(test_output_file, "w", encoding="utf-8") as f:
        json.dump(test_data, f, ensure_ascii=False, indent=4)

    print(f"Total samples: {len(data)}")
    print(f"Training samples: {len(train_data)}")
    print(f"Testing samples: {len(test_data)}")


def main():
    # Define file paths
    input_file = os.path.join("/content", "poems.json")
    train_output_file = os.path.join("/content", "train.json")
    test_output_file = os.path.join("/content", "test.json")

    # Create data directory if it doesn't exist
    os.makedirs(os.path.dirname(train_output_file), exist_ok=True)

    # Split the dataset
    split_dataset(input_file, train_output_file, test_output_file)


if __name__ == "__main__":
    main()

Total samples: 6055
Training samples: 4844
Testing samples: 1211


In [5]:
import json
import random

with open("/content/classes.json", "r", encoding="utf-8") as f:
    classes_data = json.load(f)
with open("/content/poems.json", "r", encoding="utf-8") as f:
    poems_data = json.load(f)

merged_data = classes_data + poems_data

random.shuffle(merged_data)

total_size = len(merged_data)
train_size = int(0.8 * total_size)
val_size = int(0.12 * total_size)
test_size = total_size - train_size - val_size

train_data = merged_data[:train_size]
val_data = merged_data[train_size : train_size + val_size]
test_data = merged_data[train_size + val_size :]


def write_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")


write_jsonl(train_data, "/content/train.jsonl")
write_jsonl(test_data, "/content/test.jsonl")

print(
    f"Files created: train.jsonl ({len(train_data)} entries), "
    f"val.jsonl ({len(val_data)} entries), test.jsonl ({len(test_data)} entries)"
)

Files created: train.jsonl (9708 entries), val.jsonl (1456 entries), test.jsonl (971 entries)


In [6]:
import os
import logging
from unsloth import FastModel
from datasets import load_dataset


logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

DATA_PATH = "/content/test.jsonl"
OUTS_DIR = "out"
OUTPUT_FILE = os.path.join(OUTS_DIR, "combined_metrics_results.txt")


os.makedirs(OUTS_DIR, exist_ok=True)

model, tokenizer = FastModel.from_pretrained(
    model_name="unsloth/gemma-3-4b-it",
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_8bit = False,
    full_finetuning = False,
    # token="hf_..."
)

model = FastModel.for_inference(model)

# Load test dataset
logger.info("Loading test dataset...")
test_dataset = load_dataset("json", data_files=DATA_PATH)["train"]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-04 17:35:45.000347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746380145.431579      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746380145.558112      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-04 17:36:11 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-04 17:36:12 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.1. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
!uv pip install --system --no-progress --link-mode=symlink -U evaluate rouge_score sacrebleu

Using Python 3.11.11 environment at: /usr
Resolved 43 packages in 2.12s
   Building rouge-score==0.1.2
 Downloaded aiohttp
 Downloaded lxml
      Built rouge-score==0.1.2
 Downloaded pyarrow
 Downloaded numpy
Prepared 20 packages in 1.13s
Uninstalled 16 packages in 383ms
Installed 20 packages in 143ms
 - absl-py==1.4.0
 + absl-py==2.2.2
 - aiohttp==3.11.16
 + aiohttp==3.11.18
 - certifi==2025.1.31
 + certifi==2025.4.26
 - charset-normalizer==3.4.1
 + charset-normalizer==3.4.2
 - datasets==3.5.0
 + datasets==3.5.1
 + evaluate==0.4.3
 - frozenlist==1.5.0
 + frozenlist==1.6.0
 - fsspec==2024.6.1
 + fsspec==2025.3.0
 - joblib==1.4.2
 + joblib==1.5.0
 - lxml==5.3.1
 + lxml==5.4.0
 - multidict==6.2.0
 + multidict==6.4.3
 - numpy==1.26.4
 + numpy==2.2.5
 - packaging==24.2
 + packaging==25.0
 + portalocker==3.1.1
 - pyarrow==19.0.1
 + pyarrow==20.0.0
 + rouge-score==0.1.2
 + sacrebleu==2.5.1
 - typing-extensions==4.13.1
 + typing-extensions==4.13.2
 - urllib3==2.3.0
 + urllib3==2.4.0
 - yarl==

In [8]:
import evaluate
from tqdm import tqdm

# Tokenization function
def tokenize_function(example):
    input_text = example["Instructions"] + " " + example["Input"]
    input_encoding = tokenizer(
        input_text,
        truncation=True,
        padding="max_length",
        max_length=1024,
        return_tensors="pt",
    )
    target_encoding = tokenizer(
        example["Output"],
        truncation=True,
        padding="max_length",
        max_length=1024,
        return_tensors="pt",
    )

    return {
        "input_ids": input_encoding.input_ids[0],
        "attention_mask": input_encoding.attention_mask[0],
        "labels": target_encoding.input_ids[0],
    }


test_dataset = test_dataset.map(tokenize_function)

# Load all metrics
logger.info("Loading metrics...")
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
chrf = evaluate.load("chrf")


# Function to generate predictions
def generate_predictions(model, tokenizer, dataset):
    model.eval()
    predictions = []
    references = []
    references_for_bleu = []  # BLEU expects a list of references per prediction

    logger.info("Generating predictions...")
    for example in tqdm(dataset, desc="Processing", unit="sample"):
        input_text = example["Instructions"] + " " + example["Input"]
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
        ).to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=200,
                pad_token_id=tokenizer.pad_token_id,
            )

        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predictions.append(generated_text)
        references.append(example["Output"])
        references_for_bleu.append([example["Output"]])  # Wrap in list for BLEU

    return predictions, references, references_for_bleu


# Generate outputs (do this only once for all metrics)
logger.info("Generating model outputs...")
preds, refs, refs_for_bleu = generate_predictions(model, tokenizer, test_dataset)

# Compute all metrics
logger.info("Computing metrics...")
bleu_scores = bleu.compute(predictions=preds, references=refs_for_bleu)
rouge_scores = rouge.compute(predictions=preds, references=refs)
chrf_scores = chrf.compute(predictions=preds, references=refs)

# Combine results
results = {
    "BLEU": bleu_scores["bleu"],
    "ROUGE-1": rouge_scores["rouge1"],
    "ROUGE-2": rouge_scores["rouge2"],
    "ROUGE-L": rouge_scores["rougeL"],
    "ChRF": chrf_scores["score"],
}

# Log and save results
logger.info("\nFinal Test Metrics:")
for key, value in results.items():
    logger.info(f"{key}: {value:.4f}")

with open(OUTPUT_FILE, "w") as f:
    f.write("Final Test Metrics:\n")
    for key, value in results.items():
        f.write(f"{key}: {value:.4f}\n")

logger.info(f"\nResults saved in: {OUTPUT_FILE}")

Map:   0%|          | 0/971 [00:00<?, ? examples/s]

Processing: 100%|██████████| 971/971 [5:25:00<00:00, 20.08s/sample]
